In [1]:
sample_user_ratings = [
    [5, 4, 5, 3, 3],
    [3, 2, 2, 4, 1],
    [3, 4, 3, 5, 4],
    [5, 1, 4, 2, 4],
    [2, 3, 4, 1, 1],
    [2, 3, 4, 2, 5],
]

sample_test_set = [
    (0, 0),
    (0, 3),
    (1, 1),
    (1, 4),
    (2, 0),
    (2, 4),
    (3, 2),
    (4, 1),
    (4, 3),
    (5, 0),
]

In [2]:
sample_user_ratings = [
    [5, 4, 4, -1, 5],
    [-1, 3, 5, 3, 4],
    [5, 2, -1, 2, 3],
    [-1, 2, 3, 1, 2],
    [4, -1, 5, 4, 5],
    [5, 3, -1, 3, 5],
    [3, 2, 3, 2, -1],
    [5, 3, 4, -1, 5],
    [4, 2, 5, 4, -1],
    [5, -1, 5, 3, 4]
]
sample_test_set = [(0, 4), (1, 3), (2, 3), (3, 1), (4, 2),
                   (5, 0), (6, 1), (7, 1), (8, 0), (9, 0)]

In [2]:
from predictors.least_squares import LeastSquaresPredictor
from utils import get_test_set_matrix, remove_test_set, root_mean_square_error


training_data = remove_test_set(sample_user_ratings, sample_test_set)
test_data = get_test_set_matrix(sample_user_ratings, sample_test_set)
baseline = LeastSquaresPredictor(training_data=training_data, lmda=0.25)
test_predictions = baseline.predict(sample_test_set)
all_predictions = baseline.predict_all()
rmse_training = root_mean_square_error(all_predictions, training_data)
rmse_test = root_mean_square_error(all_predictions, test_data)
print(f"{test_predictions = }")
print(f"{all_predictions = }")
print(f"{rmse_training = }")
print(f"{rmse_test = }")

test_predictions = array([4.68849898, 3.93028627, 2.12800506, 2.83108508, 4.74059504,
       4.15169449, 3.29576883, 1.4605244 , 1.99429224, 4.23431398])
all_predictions = array([[4.68849898, 3.39651842, 4.30967672, 3.93028627, 4.09959843],
       [3.41998563, 2.12800506, 3.04116336, 2.66177291, 2.83108508],
       [4.74059504, 3.44861447, 4.36177277, 3.98238232, 4.15169449],
       [3.67459109, 2.38261053, 3.29576883, 2.91637838, 3.08569054],
       [2.75250496, 1.4605244 , 2.3736827 , 1.99429224, 2.16360441],
       [4.23431398, 2.94233341, 3.85549171, 3.47610126, 3.64541343]])
rmse_training = np.float64(1.0545680887448485)
rmse_test = np.float64(1.2740136691846384)


In [ ]:
import numpy as np
from predictors.neighbor_correlations import Correlation, NeighborCorrelationsPredictor
from utils.neighbor_selection import most_similar, two_most_similar_skip_masked

baseline.prediction = np.array(
    [
        [np.nan, 2.7, 3.3, np.nan, 4.5],
        [4.1, np.nan, 3.5, 4.9, np.nan],
        [np.nan, 3.8, 2.5, 4.2, np.nan],
        [2.8, 3.1, np.nan, 2.6, 4.8],
        [3.3, np.nan, 3.7, np.nan, 2.4],
        [np.nan, 3.9, 4.0, 1.5, 3.9],
    ]
)

improved = NeighborCorrelationsPredictor(baseline=baseline, correlation=Correlation.USER)
test_predictions = improved.predict(sample_test_set, most_similar)
all_predictions = improved.predict_all(most_similar)
print(f"{test_predictions = }")
print(f"{all_predictions = }")

test_predictions = array([4.68849898, 3.93028627, 2.12800506, 2.83108508, 4.74059504,
       4.15169449, 3.29576883, 1.4605244 , 1.99429224, 4.23431398])
all_predictions = array([[nan, 2.9, 4.8, nan, 3.1],
       [1.9, nan, 1.8, 4.1, nan],
       [nan, 5. , 2.8, 5. , nan],
       [3.9, 2.9, nan, 3.5, 5. ],
       [2.2, nan, 4.2, nan, 1.3],
       [nan, 2.6, 3.7, 2.4, 5. ]])


In [ ]:
import numpy as np
from predictors.latent_factor import LatentFactorPredictor

sample_user_ratings = [
    [3, 4, 5, 3, 2, 3],
    [3, 2, 3, 4, 2, 1],
    [4, 4, 4, 5, 3, 2],
    [3, 5, 4, 4, 3, 4],
    [2, 1, 2, 2, 3, 1],
    [3, 5, 5, 4, 4, 3],
    [3, 5, 5, 3, 2, 2],
    [2, 3, 3, 2, 1, 2],
]
sample_test_set = [
    (0, 0),
    (1, 1),
    (2, 3),
    (2, 4),
    (3, 0),
    (3, 1),
    (5, 1),
    (5, 4),
    (6, 0),
    (6, 2),
    (7, 1),
    (7, 3),
]
training_data = remove_test_set(sample_user_ratings, sample_test_set)
test_data = get_test_set_matrix(sample_user_ratings, sample_test_set)
u, i = training_data.shape
k = 2
latent = LatentFactorPredictor(
    training_data=training_data,
    k=k,
    p=np.full((k, u), 1, dtype=np.float64),
    q=np.full((k, i), 1, dtype=np.float64),
    lmda=0.2,
)
latent.train(20)
all_predictions = latent.predict_all()
test_predictions = latent.predict(sample_test_set)
print(f"{all_predictions = }")
print(f"{latent.p = }")
print(f"{latent.q = }")

all_predictions = array([[2.73402602, 4.48879132, 4.30697551, 3.2186042 , 1.96912014,
        2.81803155],
       [2.83475932, 2.56022117, 2.90042499, 3.28621128, 2.69179878,
        1.50348167],
       [3.74631254, 3.7619743 , 4.11600953, 4.3521493 , 3.43987108,
        2.24331636],
       [3.38052349, 4.72745277, 4.71039242, 3.9596552 , 2.69019862,
        2.9270728 ],
       [2.17435686, 1.27615303, 1.71072411, 2.50389476, 2.27819342,
        0.68744758],
       [3.24193673, 4.89639618, 4.78844221, 3.80615816, 2.46728672,
        3.05278956],
       [2.65612574, 4.34602767, 4.1731457 , 3.12653497, 1.9176295 ,
        2.7276686 ],
       [1.77062656, 3.13429637, 2.95917071, 2.08998444, 1.20470111,
        1.97895441]])
latent.p = array([[ 0.88963584,  1.50009581,  1.87805526,  1.32698965,  1.34032657,
         1.17251361,  0.86838852,  0.51346081],
       [ 1.58503227,  0.41031614,  0.76514332,  1.47530906, -0.09021009,
         1.62844766,  1.53111621,  1.16032704]])
latent.q = arra

In [ ]:
rmse_training = root_mean_square_error(all_predictions, training_data)
rmse_test = root_mean_square_error(all_predictions, test_data)
print(f"{rmse_training = }")
print(f"{rmse_test = }")

rmse_training = np.float64(0.41866703529954136)
rmse_test = np.float64(0.6058658494466402)
